In [1]:

import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library


Solving environment: / 
  - anaconda::ca-certificates-2019.8.28-0, anaconda::openssl-1.1.1d-h1de35cc_2
  - anaconda::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-h1de35cc_2
  - anaconda::openssl-1.1.1d-h1de35cc_2, defaults::ca-certificates-2019.8.28-0
  - defaults::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-h1de35ccdone

# All requested packages already installed.



In [2]:
# for normalization 
from sklearn import preprocessing

In [3]:
df=pd.read_excel('rent_rates.xlsx') #rent_rates data was scrapped from https://www.spacelist.ca/listings/on/toronto 
df

,location,rent,space
0,For Rent - 75 Simcoe Street,"$4,000/mo",326 ft²
1,For Rent - 178 Dundas St W,"$8,000/mo","3,000 ft²"
2,100 - 76 Richmond St E,"$5,800/mo","1,450 ft²"
3,103 Richmond St E,"$3,000/mo",200 ft²
4,96 Spadina Ave,"$7,000/mo",320 ft²
...,...,...,...
171,"2657 Eglinton Ave E, Toronto, ON","$1,600/mo",450 ft²
172,"103 Orfus Rd, Toronto, ON",$15/mo,"4,610 ft²"
173,"2566 St Clair Ave W, Toronto, ON","$10,000/mo","4,022 ft²"
174,"1331 St Clair Ave W, Toronto, ON","$3,000/mo","1,200 ft²"


### cleaning the data

In [4]:
df.dropna(inplace=True)
df2=df[df['location'].str.contains('-')]
df3=df[~df['location'].str.contains('-',case=False)]
s = pd.Series(len(df2))
df2=df2.reset_index(drop=True)
df3=df3.reset_index(drop=True)

for i in range(0,len(df2)):
               df2['location'][i]=df2['location'][i].split('- ')[1] 
        
df1=pd.concat([df2,df3])

df1['rent']=df1['rent'].replace('[\$,]', '', regex=True)
df1['rent']=df1['rent'].replace('/mo', '', regex=True)

df1['space']=df1['space'].replace('Desks','25 ft²',regex=True)
df1['space']=df1['space'].replace(' ft²','',regex=True)
df1['space']=df1['space'].replace(',','',regex=True)

df1=df1.reset_index(drop=True)
df1['price_per_sqft']=df1['rent'].astype(float)/df1['space'].astype(float)
df1.head()

,location,rent,space,price_per_sqft
0,75 Simcoe Street,4000,326,12.269939
1,178 Dundas St W,8000,3000,2.666667
2,76 Richmond St E,5800,1450,4.000000
3,600 Bay St,6300,1700,3.705882
4,717 Queen St E,1500,400,3.750000


### finding the longitude and latitude addresses

In [5]:
df1['latitude'] = ""
df1['longitude']=""

for i in range(0,len(df1)):
    address = df1['location'][i]+ ', Toronto, canada'
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    df1['latitude'][i]= location.latitude
    df1['longitude'][i] = location.longitude
    
    



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [6]:
rent_spaces=df1

In [7]:
address = 'Toronto, Ontario' ## the city for the business

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


In [8]:
#normalize the rent per sqft to the range (1-10)
x=rent_spaces[['price_per_sqft']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(1, 10))
# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)
rent_spaces['price_per_sqft_norm']=x_scaled

In [9]:
# add the rent spaces to the location
# add markers to map
map_T = folium.Map(location=[latitude, longitude], zoom_start=11)
#folium.GeoJson(BIA.geometry).add_to(map_T)
for lat, lng, name,n in zip(rent_spaces['latitude'], rent_spaces['longitude'], rent_spaces['location'],rent_spaces['price_per_sqft_norm']):
        label = folium.Popup(name, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=n,
        popup=label,
        color='red',
        fill=True,
        fill_opacity=0.1,
        line_opacity=0.1,
        parse_html=False).add_to(map_T)  
    
map_T